In [138]:
import pandas as pd
import numpy as np
import wget

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 50)

import xgboost as xgb
import sklearn

print("XGB Version          : ", xgb.__version__)
print("Scikit-Learn Version : ", sklearn.__version__)

XGB Version          :  2.1.2
Scikit-Learn Version :  1.4.2


In [139]:
data0 = pd.read_csv("cleaned_verified_online.csv")
data0.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,8855181,https://sparkflaretrust.com/connect.html,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:46:02+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other
1,8855180,https://llltreuirzleeno.webflow.io/,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:37:15+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other
2,8855179,http://llltreuirzleeno.webflow.io,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:37:14+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other
3,8855176,http://hubtrezorstart.webflow.io,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:32:28+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other
4,8855177,https://hubtrezorstart.webflow.io/,http://www.phishtank.com/phish_detail.php?phis...,2024-11-16T03:32:28+00:00,yes,2024-11-16T03:52:29+00:00,yes,Other


In [140]:
data0.shape


(67678, 8)

In [141]:
phishurl = data0.sample(n = 10000, random_state = 42).copy()
phishurl = phishurl.reset_index(drop=True)
phishurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,8719230,https://encremoinschere.com/405.php,http://www.phishtank.com/phish_detail.php?phis...,2024-08-17T19:23:40+00:00,yes,2024-08-17T19:42:55+00:00,yes,Other
1,8737793,https://dk11863382937.activehosted.com/f/1,http://www.phishtank.com/phish_detail.php?phis...,2024-08-30T09:17:43+00:00,yes,2024-08-30T09:33:11+00:00,yes,Other
2,8751044,https://jemi.so/maoulika41868678871671,http://www.phishtank.com/phish_detail.php?phis...,2024-09-09T17:45:52+00:00,yes,2024-09-09T17:52:59+00:00,yes,Other
3,8787374,https://geameinilogin.webflow.io/,http://www.phishtank.com/phish_detail.php?phis...,2024-09-29T19:19:16+00:00,yes,2024-09-29T19:22:55+00:00,yes,Other
4,8396571,https://re-register-device.firebaseapp.com/,http://www.phishtank.com/phish_detail.php?phis...,2023-12-18T08:23:31+00:00,yes,2023-12-18T08:33:54+00:00,yes,Other


In [142]:
phishurl.shape

(10000, 8)

In [264]:
#legitimate URL
data1 = pd.read_csv("Benign_list_big_final.csv")
data1.columns = ['URLs']
data1.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [287]:
legiurl = data1.sample(n = 10000, random_state = 42).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

In [289]:
legiurl.shape

(10000, 1)

In [146]:
# importing required packages for this section
from urllib.parse import urlparse,urlencode
import ipaddress
import re

In [150]:
# 1.Domain of the URL (Domain) 
def getDomain(url):  
  domain = urlparse(url).netloc
  if re.match(r"^www.",domain):
    domain = domain.replace("www.","")
    print(domain)
  return domain

In [210]:
# 2.Checks for IP address in URL (Have_IP)
def havingIP(url):
  ip_address_match = re.search(r'\d+\.\d+\.\d+\.\d+', url)
  if ip_address_match:
    ip_address = ip_address_match.group()
    try:
      ipaddress.ip_address(ip_address)
      ip = 1  # Valid IP address
    except:
      ip = 0  # Invalid IP address
    return ip
  else:
    # No IP address found in the URL
    return 0

In [152]:
# 3.Checks the presence of @ in URL (Have_At)
def haveAtSign(url):
  if "@" in url:
    print('Have_At')
    at = 1    
  else:
    at = 0    
  return at

In [14]:
# 4.Finding the length of URL and categorizing (URL_Length)
def getLength(url):
  if len(url) < 54:
    length = 0            
  else:
    length = 1            
  return length

In [15]:
# 5.Gives number of '/' in URL (URL_Depth)
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

In [16]:
# 6.Checking for redirection '//' in the url (Redirection)
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

In [159]:
# 7.Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

In [160]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [166]:
# 8. Checking for Shortening Services in URL (Tiny_URL)
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

In [165]:
# 9.Checking for Prefix or Suffix Separated by (-) in the Domain (Prefix/Suffix)
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate
     

In [21]:
!pip install python-whois


In [22]:
# importing required packages for this section
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
import csv
from functools import lru_cache
from datetime import datetime


In [23]:
# 11.DNS Record availability (DNS_Record)
# obtained in the featureExtraction function itself

In [182]:
# 12.Web traffic (Web_Traffic)
def web_traffic(url, csv_file):
    # Read CSV data into a dictionary for efficient lookup
    rank_data = {}
    try:
      with open(csv_file, 'r') as csvfile:
          reader = csv.reader(csvfile)
          for row in reader:
              if len(row) >= 2:
                  rank_data[row[1]] = int(row[0])
              else:
                  print(f"Warning: Invalid row in CSV: {row}")
    except FileNotFoundError:
      print(f"CSV file not found: {csv_file}")
      return 0
    except csv.Error as e:
      print(f"Error reading CSV file: {e}")
      return 0

    # Extract domain from URL
    domain = url.split("//")[1].split("/")[0]
    print(domain)
    # Check if domain exists in the CSV
    if domain in rank_data:
      rank = rank_data[domain]
      print('rank', rank)
      if rank < 100000:
        return 0
      else:
        return 1
    else:
      # Domain not found in CSV, return 1 if rank is assumed to be low
      return 1

In [184]:
# 12.Web traffic (Web_Traffic)
def check_rank(url):
  try:
    #Filling the whitespaces in the URL if any
    url = urllib.parse.quote(url)
    rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
    rank = int(rank)
  except TypeError:
        return 1
  if rank <100000:
    return 1
  else:
    return 0

In [26]:
# 13.Survival time of domain: The difference between termination time and creation time (Domain_Age)  
def domainAge(domain_name):
  creation_date = domain_name.creation_date
  expiration_date = domain_name.expiration_date
  if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if ((expiration_date is None) or (creation_date is None)):
      return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
      return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
    if ((ageofdomain/30) < 6):
      age = 1
    else:
      age = 0
  return age

In [246]:
# 14.End time of domain: The difference between termination time and current time (Domain_End) 
# def domainEnd(domain_name):
#   expiration_date = domain_name.expiration_date
#   if isinstance(expiration_date,str):
#     try:
#       expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
#     except:
#       return 1
#   if (expiration_date is None):
#       return 1
#   elif (type(expiration_date) is list):
#       return 1
#   else:
#     today = datetime.now()
#     end = abs((expiration_date - today).days)
#     if ((end/30) < 6):
#       end = 0
#     else:
#       end = 1
#   return end

import datetime
import whois

def domainEnd(domain_name):
    try:
        expiration_date = whois.whois(domain_name).expiration_date

        # Ensure expiration_date is not None and is a list/tuple (some whois lookups return like that)
        if expiration_date is not None:
            if isinstance(expiration_date, (list, tuple)):
                expiration_date = expiration_date[0]  # Use first date if multiple

            # Make 'today' timezone-aware by setting its timezone to UTC
            today = datetime.datetime.now(datetime.timezone.utc)
            end = abs((expiration_date - today).days)
            if ((end/30) < 6):
                end = 0
            else:
                end = 1
    except:
        end = 1

    return end

In [247]:
#HTML and JavaScript based Features
import requests

In [274]:
# 15. IFrame Redirection (iFrame)
# def iframe(response):
#   if response == "":
#       return 1
#   else:
#       if re.findall(r"[<iframe>|<frameBorder>]", response.text):
#           return 0
#       else:
#           return 1
import signal
import re

def timeout_handler(signum, frame):
    raise TimeoutError("Timeout")

def iframe(response, timeout=20):
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout)  # Set the timeout in seconds

    try:
        if response == "":
            return 1
        else:
            if re.findall(r"[<iframe>|<frameBorder>]", response.text):
                return 0
            else:
                return 1
    except TimeoutError:
        return 1  # Or handle the timeout as needed
        print("Timeout occurred during iframe detection")
    finally:
        signal.alarm(0)

In [192]:
# 16.Checks the effect of mouse over on status bar (Mouse_Over)
def mouseOver(response): 
  if response == "" :
    return 1
  else:
    if re.findall("", response.text):
      return 1
    else:
      return 0

In [193]:
# 17.Checks the status of the right click attribute (Right_Click)
def rightClick(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0
    else:
      return 1

In [195]:
# 18.Checks the number of forwardings (Web_Forwards)    
def forwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
      return 1

In [243]:
!pip install diskcache

In [242]:
# @title Default title text
import whois
from urllib.parse import urlparse
import diskcache as dc

cache = dc.Cache("whois_cache")  # Create or load a cache

def cached_whois_lookup(domain):
    """
    Perform a WHOIS lookup with persistent caching.
    :param domain: The domain name to query.
    :return: WHOIS data as a dictionary or None if the lookup fails.
    """
    if domain in cache:
        return cache[domain]
    try:
        whois_data = whois.whois(domain)
        cache[domain] = whois_data  # Store in cache
        return whois_data
    except:
        cache[domain] = None  # Cache the failure
        return None

In [224]:
#Function to extract features
def featureExtraction(url,label):
  #print('url', url,'label', label)
  features = []
  #Address bar based features (10)
  features.append(getDomain(url))
  features.append(havingIP(url))
  features.append(haveAtSign(url))
  features.append(getLength(url))
  features.append(getDepth(url))
  features.append(redirection(url))
  features.append(httpDomain(url))
  features.append(tinyURL(url))
  features.append(prefixSuffix(url))
  
  #Domain based features (4)
  dns = 0
  try:
    domain_name = whois.whois(urlparse(url).netloc)
    #print(domain_name)
  except:
    dns = 1

  features.append(dns)
  # features.append(web_traffic(url))
  features.append(web_traffic(url,'top-1m.csv'))
  features.append(1 if dns == 1 else domainAge(domain_name))
  features.append(1 if dns == 1 else domainEnd(domain_name))
  
  #HTML & Javascript based features (4)
  try:
    response = requests.get(url,timeout=5)  
  except:
    response = ""
  features.append(iframe(response))
  features.append(mouseOver(response))
  features.append(rightClick(response))
  features.append(forwarding(response))
  features.append(label)
  
  return features

In [299]:
#Legitimate URLs
legiurl.shape

AttributeError: 'DataFrame' object has no attribute 'greet'

In [291]:
#Extracting the feautres & storing them in a list
#import cProfile
from IPython.display import clear_output

legi_features = []
label = 0

for i in range(0, 1000):
    url = legiurl['URLs'][i]
    print(url,i)
    #cProfile.run("legi_features.append(featureExtraction(url,label))")
    legi_features.append(featureExtraction(url,label))
    #clear_output(wait=True)

https://medium.com/cuepoint/kanye-west-and-childish-gambino-s-advanced-web-marketing-techniques-cb9c139d7a3b 999
{
  "domain_name": "MEDIUM.COM",
  "registrar": "Amazon Registrar, Inc.",
  "registrar_url": [
    "http://registrar.amazon.com",
    "https://registrar.amazon.com"
  ],
  "reseller": null,
  "whois_server": "whois.registrar.amazon",
  "referral_url": null,
  "updated_date": "2024-04-21 22:35:12",
  "creation_date": "1998-05-27 04:00:00",
  "expiration_date": "2025-05-26 04:00:00",
  "name_servers": [
    "ALINA.NS.CLOUDFLARE.COM",
    "KIP.NS.CLOUDFLARE.COM"
  ],
  "status": [
    "clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited",
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
    "clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited"
  ],
  "emails": [
    "trustandsafety@support.aws.com",
    "e21a9e56-5a30-40f3-8881-743a5e9f0ad8@identity-protect.org"
  ],
  "dnssec": "unsigned",
  "name": "On behalf of me

In [292]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic', 
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

legitimate = pd.DataFrame(legi_features, columns= feature_names)
legitimate.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,correios.com.br,0,0,1,4,0,0,0,0,0,0,1,1,0,0,1,0,0
1,caixa.gov.br,0,0,1,6,0,0,0,0,0,0,1,1,0,0,1,0,0
2,olx.ua,0,0,1,4,0,0,0,0,0,0,1,1,0,0,1,1,0
3,emgn.com,0,0,1,2,0,0,0,0,0,1,1,1,1,1,1,1,0
4,metro.co.uk,0,0,1,4,0,0,0,0,0,0,0,1,0,0,1,0,0


In [293]:
legitimate.to_csv('legitimate.csv', index= False)

In [240]:
phishurl.shape

(10000, 8)

In [248]:
#Extracting the feautres & storing them in a list
from IPython.display import clear_output

phish_features = []
label = 1
for i in range(0, 10000):
  url = phishurl['url'][i]
  print(url,i)
  phish_features.append(featureExtraction(url,label))
  clear_output(wait=True)

https://pub-6e3d23a65d6b4c24913d2334c820565d.r2.dev/linkSEN.html 9999
{
  "domain_name": "r2.dev",
  "registrar": "CloudFlare, Inc.",
  "registrar_url": [
    "None",
    "https://www.cloudflare.com"
  ],
  "reseller": null,
  "whois_server": "whois.cloudflare.com",
  "referral_url": null,
  "updated_date": [
    "2023-09-04 18:27:26",
    "2024-11-21 06:04:36"
  ],
  "creation_date": "2022-08-23 14:38:38",
  "expiration_date": "2025-08-23 14:38:38",
  "name_servers": [
    "camilo.ns.cloudflare.com",
    "raphaela.ns.cloudflare.com"
  ],
  "status": [
    "clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited",
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
    "clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited"
  ],
  "emails": "registrar-abuse@cloudflare.com",
  "dnssec": "unsigned",
  "name": [
    "REDACTED FOR PRIVACY",
    "DATA REDACTED"
  ],
  "org": "DATA REDACTED",
  "address": [
    "REDACTED FOR PRIVACY",
    

In [251]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic', 
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

phishing = pd.DataFrame(phish_features, columns= feature_names)
phishing.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,encremoinschere.com,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,1
1,dk11863382937.activehosted.com,0,0,0,2,0,0,0,0,0,1,1,1,0,0,1,0,1
2,jemi.so,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1
3,geameinilogin.webflow.io,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,1
4,re-register-device.firebaseapp.com,0,0,0,0,0,0,0,1,1,1,1,1,0,0,1,0,1


In [252]:
# Storing the extracted legitimate URLs fatures to csv file
phishing.to_csv('phishing.csv', index= False)

In [253]:
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)
urldata.head()


,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,correios.com.br,0,0,1,4,0,0,0,0,0,0,1,1,0,0,1,0,0
1,caixa.gov.br,0,0,1,6,0,0,0,0,0,0,1,1,0,0,1,0,0
2,olx.ua,0,0,1,4,0,0,0,0,0,0,1,1,0,0,1,1,0
3,emgn.com,0,0,1,2,0,0,0,0,0,1,0,1,1,1,1,1,0
4,metro.co.uk,0,0,1,4,0,0,0,0,0,0,0,1,0,0,1,0,0


In [254]:
urldata.shape

(17305, 18)

In [255]:
# Storing the data in CSV file
urldata.to_csv('urldata.csv', index=False)
